In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from pytrack_analysis import Multibench
from pytrack_analysis.dataio import RawData, get_session_list
from pytrack_analysis.profile import get_profile, get_scriptname, show_profile
from pytrack_analysis.posttracking import frameskips, get_displacements, get_head_tail, mistracks, get_pixel_flip
from pytrack_analysis.viz import plot_along, plot_fly, plot_intervals, plot_overlay, plot_ts

E:/Dennis/Google Drive/PhD Project/Tracking Analysis/profile\profile.yaml VERO-PC Windows_NT


E:\Users\Vero\AppData\Local\Programs\Python\Python36\Lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
experiment = 'DIFF'
user = 'degoldschmidt'
ascript = '001-post_tracking'

In [4]:
profile = get_profile(experiment, user, script=ascript)
load_from_file = True
folders = profile.get_folders()

System 'VERO-PC' found.
Project 'DIFF' found.


In [5]:
### load video session data and metadata
this_session = 1
colnames = ['datetime', 'elapsed_time', 'frame_dt', 'body_x',   'body_y',   'angle',    'major',    'minor']
colunits = ['Datetime', 's',            's',        'px',       'px',       'rad',      'px',       'px']
session_data = RawData(experiment, this_session, folders, columns=colnames, units=colunits, noVideo=False)


Start post-tracking analysis for video session: 01
[pytrack_analysis.dataio]	Timestamp: Friday, 24. November 2017 08:26
[pytrack_analysis.dataio]	Loading raw data...done.
[pytrack_analysis.arena]	loading geometry data...done.
[pytrack_analysis.food_spots]	loading food spots data...found a total of 48 spots...done.


In [6]:
### scale trajectories to mm
scale = 8.543
session_data.set_scale('fix_scale', scale, unit='mm')
### detect frameskips
frameskips(session_data, dt='frame_dt')

[pytrack_analysis.posttracking]	detected frameskips: 17 (0.016% of all frames)


In [7]:
for i_arena, each_df in enumerate(session_data.raw_data):
    arena = session_data.arenas[i_arena]
    ### compute frame-to-frame displacements
    each_df['displacement'] = get_displacements(each_df, x='body_x', y='body_y')
    ### detect mistracked frames
    each_df = mistracks(each_df, i_arena, dr='displacement', major='major', thresholds=(4, 5))
    ### compute head and tail positions
    head_tails = get_head_tail(each_df, x='body_x', y='body_y', angle='angle', major='major')
    each_df['head_x'] = head_tails['head_x']
    each_df['head_y'] = head_tails['head_y']
    each_df['head_px'] = scale * head_tails['head_x'] + arena.x
    each_df['head_py'] = scale * head_tails['head_y'] + arena.y
    each_df['tail_px'] = scale * head_tails['tail_x'] + arena.x
    each_df['tail_py'] = scale * head_tails['tail_y'] + arena.y
    ### detect head flips
    file_id = 4 * (this_session-1) + i_arena + 1
    _file = os.path.join(folders['processed'],'post_tracking','{}_{:03d}.csv'.format(experiment, file_id))
    if load_from_file:
        print("Loading data from", _file)
        df = pd.read_csv(_file, index_col='frame')
        each_df['flip'] = df['flip']
        each_df['headpx'] = df['headpx']
        each_df['tailpx'] = df['tailpx']
    else:
        flip, headpx, tailpx = get_pixel_flip(each_df, hx='head_px', hy='head_py', tx='tail_px', ty='tail_py', video=session_data.video_file, start=session_data.first_frame)
        each_df['flip'] = flip
        each_df['headpx'] = headpx
        each_df['tailpx'] = tailpx
        print("Saving data to", _file)
        each_df.to_csv(_file, index_label='frame')
session_data.flip_y()

[pytrack_analysis.posttracking]	Arena ↖ - mistracked frames: 0
Loading data from E:/Dennis/Google Drive/PhD Project/Experiments\001-DifferentialDeprivation\data\processed\post_tracking\DIFF_001.csv
[pytrack_analysis.posttracking]	Arena ↗ - mistracked frames: 0
Loading data from E:/Dennis/Google Drive/PhD Project/Experiments\001-DifferentialDeprivation\data\processed\post_tracking\DIFF_002.csv
[pytrack_analysis.posttracking]	Arena ↙ - mistracked frames: 0
Loading data from E:/Dennis/Google Drive/PhD Project/Experiments\001-DifferentialDeprivation\data\processed\post_tracking\DIFF_003.csv
[pytrack_analysis.posttracking]	Arena ↘ - mistracked frames: 0
Loading data from E:/Dennis/Google Drive/PhD Project/Experiments\001-DifferentialDeprivation\data\processed\post_tracking\DIFF_004.csv


In [8]:
i = 0
arena = session_data.arenas[i]
nintervals = 24
f, ax = plt.subplots()
ax = plot_interval(0, session_data.raw_data[i], x='body_x', y='body_y', hx='head_x', hy='head_y', flip='flip', time='elapsed_time', arena=arena, spots=arena.spots, ncols=3)
#_file = os.path.join(folders['out'], '{}_{:03d}_intervals.pdf'.format(args.exp, file_id))
plt.tight_layout()
plt.show()
    #f.savefig(_file, dpi=600)

NameError: name 'plot_interval' is not defined